In [1]:
!pip install scipy  tiktoken retry loguru

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import os
from pprint import pprint
import re
from tqdm import tqdm
import random

import uuid
import openai
from openai import OpenAI
import tiktoken
import json
import numpy as np
import requests
from scipy import sparse
#from rank_bm25 import BM25Okapi
#import jieba
from http import HTTPStatus


from concurrent.futures import ThreadPoolExecutor, as_completed
from loguru import logger
import json
import time
from tqdm import tqdm

logger.remove()  # 移除默认的控制台输出
logger.add("logs/app_{time:YYYY-MM-DD}.log", level="INFO", rotation="00:00", retention="10 days", compression="zip")

MODEL_NAME = 'Qwen2-7B-Instruct-lora'

In [3]:
def api_retry(MODEL_NAME, query):
    max_retries = 5
    retry_delay = 60  # in seconds
    attempts = 0
    while attempts < max_retries:
        try:
            return call_qwen_api(MODEL_NAME, query)
        except Exception as e:
            attempts += 1   
            if attempts < max_retries:
                logger.warning(f"Attempt {attempts} failed for text: {query}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                logger.error(f"All {max_retries} attempts failed for text: {query}. Error: {e}")
                raise

In [4]:
def call_qwen_api(MODEL_NAME, query):
    # 这里采用dashscope的api调用模型推理，通过http传输的json封装返回结果
    
    client = OpenAI(
        base_url="http://localhost:8000/v1",
        api_key="sk-xxx", # 随便填写，只是为了通过接口参数校验
    )
    completion = client.chat.completions.create(
      model=MODEL_NAME,
      messages=[
                # {'role':'system','content':'你是一个解决推理任务的专家，你需要分析出问题中的每个实体以及响应关系。然后根据问题一步步推理出结果。并且给出正确的结论。'},

        {"role": "user", "content": query}
      ]
    )
    return completion.choices[0].message.content

In [5]:
# call_qwen_api(MODEL_NAME,'你好')

In [6]:
# 这里定义了prompt推理模版

def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""
    # print(prompt)
    return prompt


In [13]:
# 这里使用extract抽取模获得抽取的结果

def extract(input_text):
    ans_pattern = re.compile(r"(.)", re.S)
    # ans_pattern = re.compile(r"答案是：(.)", re.S)

    problems = ans_pattern.findall(input_text)
    # print(problems)
    if(problems == ''):
        return 'A'
    return problems[0]

In [8]:
def process_datas(datas,MODEL_NAME):
    results = []
    with ThreadPoolExecutor(max_workers=16) as executor:
        future_data = {}
        lasttask = ''
        lastmark = 0
        lens = 0
        # 送入多线程任务
        for data in tqdm(datas, desc="Submitting tasks", total=len(datas)):
            problem = data['problem']
            for id,question in enumerate(data['questions']):
                prompt = get_prompt(problem, 
                                    question['question'], 
                                    question['options'],
                                    )

                future = executor.submit(api_retry, MODEL_NAME, prompt)
                
                future_data[future] = (data,id)
                time.sleep(0.6)  # 控制每0.6秒提交一个任务
                lens += 1
        # 处理多线程任务
        for future in tqdm(as_completed(future_data), total=lens, desc="Processing tasks"):
            # print('data',data)
            data = future_data[future][0]
            problem_id = future_data[future][1]
            try:
                res  = future.result()
                extract_response = extract(res)
                # print('res',extract_response)
                data['questions'][problem_id]['answer'] = extract_response
                results.append(data)
                # print('data',data)
                
            except Exception as e:
                logger.error(f"Failed to process text: {data}. Error: {e}")
    
    return results

In [9]:
def process_datas(datas,MODEL_NAME):
    results = []

    # 送入多线程任务
    for data in tqdm(datas, desc="Submitting tasks", total=len(datas)):
        problem = data['problem']
        for id,question in enumerate(data['questions']):
            prompt = get_prompt(problem, 
                                question['question'], 
                                question['options'],
                                    )
            res = api_retry(MODEL_NAME, prompt)
            extract_response = extract(res)
            data['questions'][id][MODEL_NAME] = extract_response
            results.append(data) 
    return results

In [10]:
def main(ifn, ofn):
    if os.path.exists(ofn):
        pass
    data = []
    # 按行读取数据
    with open(ifn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)
    datas = data[:10]
    # print(data)
    # 均匀地分成多个数据集
    return_list = process_datas(datas,MODEL_NAME)
    print(len(return_list))
    print("All tasks finished!")
    return return_list

In [11]:
def evaluate(data):
    data = data

    pse = 0
    cnt = 0
    tot = 0
    for task in data:
        for question in task['questions']:
            print(task)
            if MODEL_NAME in question:
                
                tot += 1
                cnt += question[MODEL_NAME] == question['answer']
            else:
                pse += 1

    print(cnt, tot, cnt/tot, pse)

In [ ]:
if __name__ == '__main__':

    a = extract("""根据欧几里得算法，逐步解析计算两个数6和7的最大公约数（gcd）的步骤如下：

1. 判断6和7是否相等：不相等。
2. 判断6和7大小关系，7 > 6，所以用更大的数7减去较小的数6得到结果1。
3. 现在计算6和1的最大公约数。
4. 6 > 1，根据算法用更大的数6减去较小的数1得到结果5。
5. 再计算5和1的最大公约数。
6. 5 > 1，用5减去1得到结果4。
7. 再计算4和1的最大公约数。
8. 4 > 1，用4减去1得到结果3。
9. 再计算3和1的最大公约数。
10. 3 > 1，用3减去1得到结果2。
11. 再计算2和1的最大公约数。
12. 2 > 1，用2减去1得到结果1。
13. 最后计算1和1的最大公约数，两数相等，gcd即为这两个数，也就是1。

因此，6和7的最大公约数是1。

答案是：C.""")

    print(a)
    return_list = main('round1_train_data.jsonl', 'upload_test.jsonl')


根


Submitting tasks: 100%|██████████| 10/10 [00:03<00:00,  2.52it/s]

28
All tasks finished!


In [ ]:
return_list

[{'problem': '有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：',
  'questions': [{'question': '选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：',
    'options': ['le chat saute sur la lune',
     'le chat sauts sur le lune',
     'le sauts chat sur le lune',
     'le chat sauts sur le lune'],
    'answer': 'D',
    'Qwen2-7B-Instruct-lora': 'A'},
   {'question': '选择题 2：\n英文句子 "the cow plays the fiddle" 翻译成法文是：',
    'options': ['le vache jouer le violon',
     'le jouer vache le violon',
     'le vache jouer la vièle',
     'la vache joue le violon'],
    'answer': 'A',
    'Qwen2-7B-Instruct-lora': 'A'},
   {'question': '选择题 3：\n英文句子 "the egg falls off the wall" 翻译成法文是：',
    'options': ['le bougre des chutes de le mur',
     'le oeuf tombe de le mur',
     'le bougre tomb

In [ ]:
def has_complete_answer(questions):
    # 这里假设完整答案的判断逻辑是：每个question都有一个'answer'键
    for question in questions:
        if MODEL_NAME not in question:
            return False
    return True

def filter_problems(data):
    result = []
    problem_set = set()

    for item in data:
        # print('处理的item' ,item)
        problem = item['problem']
        if problem in problem_set:
            # 找到已存在的字典
            for existing_item in result:
                if existing_item['problem'] == problem:
                    # 如果当前字典有完整答案，替换已存在的字典
                    if has_complete_answer(item['questions']):
                        existing_item['questions'] = item['questions']
                        existing_item['id'] = item['id']
                    break
        else:
            # 如果当前字典有完整答案，添加到结果列表
            if has_complete_answer(item['questions']):
                result.append(item)
                problem_set.add(problem)

    return result

In [ ]:
return_list
return_list = filter_problems(return_list)
sorted_data = sorted(return_list, key=lambda x: int(str(x['id'])[-3:]))
print(sorted_data)

[{'problem': '有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：', 'questions': [{'question': '选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：', 'options': ['le chat saute sur la lune', 'le chat sauts sur le lune', 'le sauts chat sur le lune', 'le chat sauts sur le lune'], 'answer': 'D', 'Qwen2-7B-Instruct-lora': 'A'}, {'question': '选择题 2：\n英文句子 "the cow plays the fiddle" 翻译成法文是：', 'options': ['le vache jouer le violon', 'le jouer vache le violon', 'le vache jouer la vièle', 'la vache joue le violon'], 'answer': 'A', 'Qwen2-7B-Instruct-lora': 'A'}, {'question': '选择题 3：\n英文句子 "the egg falls off the wall" 翻译成法文是：', 'options': ['le bougre des chutes de le mur', 'le oeuf tombe de le mur', 'le bougre tomber de le mur', 'des chutes le bougre de le mur'], 'answer': 'A', 'Qwen2-7B-I

In [ ]:
def find_missing_ids(dict_list):
    # 提取所有序号
    extracted_ids = {int(d['id'][-3:]) for d in dict_list}
    
    # 创建0-500的序号集合
    all_ids = set(range(10))
    
    # 找出缺失的序号
    missing_ids = all_ids - extracted_ids
    
    return sorted(missing_ids)

# 示例字典列表
dict_list = sorted_data

# 找出缺失的序号
missing_ids = find_missing_ids(dict_list)
print("缺失的序号:", missing_ids)

缺失的序号: []


In [ ]:
len(missing_ids)

0

In [ ]:
data  = []
with open('round1_test_data.jsonl') as reader:
    for id,line in enumerate(reader):
        if(id in missing_ids):
            sample = json.loads(line)
            for question in sample['questions']:
                question['answer'] = 'A'
            sorted_data.append(sample)
sorted_data = sorted(sorted_data, key=lambda x: int(str(x['id'])[-3:]))
        

In [ ]:
with open('upload_test.jsonl', 'w') as writer:
    for sample in sorted_data:
        writer.write(json.dumps(sample, ensure_ascii=False))
        writer.write('\n')

In [ ]:
evaluate(sorted_data)

{'problem': '有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：', 'questions': [{'question': '选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：', 'options': ['le chat saute sur la lune', 'le chat sauts sur le lune', 'le sauts chat sur le lune', 'le chat sauts sur le lune'], 'answer': 'D', 'Qwen2-7B-Instruct-lora': 'A'}, {'question': '选择题 2：\n英文句子 "the cow plays the fiddle" 翻译成法文是：', 'options': ['le vache jouer le violon', 'le jouer vache le violon', 'le vache jouer la vièle', 'la vache joue le violon'], 'answer': 'A', 'Qwen2-7B-Instruct-lora': 'A'}, {'question': '选择题 3：\n英文句子 "the egg falls off the wall" 翻译成法文是：', 'options': ['le bougre des chutes de le mur', 'le oeuf tombe de le mur', 'le bougre tomber de le mur', 'des chutes le bougre de le mur'], 'answer': 'A', 'Qwen2-7B-In